In [2]:
import os
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
from dotenv import load_dotenv

In [4]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter

In [11]:
# doc_reader = PyPDFLoader("The_3_Alarms.pdf")
doc_reader = PyPDFLoader("doc.pdf")
doc_reader

AttributeError: 'str' object has no attribute 'lazy_load'

In [8]:
pages = doc_reader.load_and_split()
pages[0]

Document(page_content='Amrita Vishwa Vidyapeetham \nAnticipated Questions from NAAC Peer Review Team (PRT) \n \nCurricular aspects \n1. In what way are you ensuring that the curriculum design is aligned with the institutional \ngoals and objectives? Can you provide some examples? \n2. Do you have examples to show that the curriculum addresses the current \nlocal/regional/national/global needs of stakeholders? \n3. Do you have any courses related to gender, environment, sustainability, ethics & values? \n4. In what way do you ensure that employability is a significant consideration in developing \nthe curriculum? \n5. In what way does the curriculum include value-based education? \n6. Are all the department faculty members involved in providing inputs for curriculum \ndevelopment? How is that done? Are there documents to show that the inputs are \ncaptured? \n7. Do you have a mechanism for formally collecting inputs/feedback on curriculum design \n& development from industrial employers

In [9]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

In [14]:
texts = text_splitter.split_documents(pages)
print(texts[0])

page_content='Amrita Vishwa Vidyapeetham \nAnticipated Questions from NAAC Peer Review Team (PRT) \n \nCurricular aspects \n1. In what way are you ensuring that the curriculum design is aligned with the institutional \ngoals and objectives? Can you provide some examples? \n2. Do you have examples to show that the curriculum addresses the current \nlocal/regional/national/global needs of stakeholders? \n3. Do you have any courses related to gender, environment, sustainability, ethics & values? \n4. In what way do you ensure that employability is a significant consideration in developing \nthe curriculum? \n5. In what way does the curriculum include value-based education? \n6. Are all the department faculty members involved in providing inputs for curriculum \ndevelopment? How is that done? Are there documents to show that the inputs are \ncaptured? \n7. Do you have a mechanism for formally collecting inputs/feedback on curriculum design \n& development from industrial employers and acad